<h1 style='text-align: center;'> Sewage and Drainage </h1>
<h4 style='text-align: justify;'> BoSung Kim </h3>

Github: https://github.com/kimbos0523/Sewer-and-Drainage-Infrastructure-in-Boston </br>
Webpage: https://pages.github.khoury.northeastern.edu/kimbos/ds4200group8/</br>
Demo Video: https://drive.google.com/drive/folders/1j_YGhLT_TTIs9vfo1CK4E416knu1V3rK </br>
Final Presentation: https://drive.google.com/drive/folders/1j_YGhLT_TTIs9vfo1CK4E416knu1V3rK

In [1]:
import pandas as pd
import altair as alt

# load the csv data
df_sso = pd.read_csv('SSO Locations 1 Year_cleaned.csv')
df_ls = pd.read_csv('Lead Services_cleaned.csv')
df_rain_year = pd.read_csv('BWSC_Rainfall_Yearly_2022.csv')
df_rain_month = pd.read_csv('BWSC_Rainfall_Monthly_2022.csv')

# Sort df_sso by 1. Start Date, 2. Start Time
df_sso = df_sso.sort_values(by=['Start Date', 'Start Time'], key=pd.to_datetime)

df_sso['Start Time'] = pd.to_datetime(df_sso['Start Time'])
df_sso['End Time'] = pd.to_datetime(df_sso['End Time'])

In [2]:
# Overview the data structure
df_sso.head(5)

,SSO Location,Neighborhood,Spill Type,Start Date,Start Time,End Date,End Time,Spill Volume,Cause,Corrective Action,Latitude,Longitude
26,254 SUMMER ST,SOUTH BOSTON,PUBLIC,10/18/2021,2022-12-05 09:12:00,10/18/2021,2022-12-05 11:45:00,30,FOG,FURTHER INVESTIGATION REQUIRED,42.390062,-71.113337
7,1455 COMMONWEALTH AV,ALLSTON/BRIGHTON,PRIVATE - 100 GALLONS OR MORE,10/23/2021,2022-12-05 19:52:00,10/24/2021,2022-12-05 14:30:00,1000,PRIVATE LATERAL,OWNER RESPONSIBILITY,42.348773,-71.141198
8,1455 COMMONWEALTH AV,ALLSTON/BRIGHTON,PRIVATE - NOT CONTAINED TO BUILDING,10/23/2021,2022-12-05 19:52:00,10/24/2021,2022-12-05 14:30:00,1000,PRIVATE LATERAL,OWNER RESPONSIBILITY,42.348773,-71.141198
2,125 NASHUA ST,CENTRAL,PUBLIC,12/14/2021,2022-12-05 09:51:00,12/14/2021,2022-12-05 11:35:00,50,DEBRIS,FURTHER INVESTIGATION REQUIRED,42.367100,-71.064000
43,512 CENTRE ST,JAMAICA PLAIN,PUBLIC,12/15/2021,2022-12-05 10:00:00,12/15/2021,2022-12-05 12:15:00,300,DEBRIS,FURTHER INVESTIGATION REQUIRED,42.317590,-71.112659


In [3]:
# Basic url and topo features to represent Boston Map
boston_url = "https://raw.githubusercontent.com/lsouth/DS4200/main/Boston_Neighborhoods.json"

boston = alt.topo_feature(boston_url, feature='Boston_Neighborhoods')

In [4]:
# Boston background map
background = alt.Chart(boston, title="Sanitary Sewer Overflow Map").mark_geoshape(
    fill="lightgray",
    stroke="white"
).properties(
    width=400,
    height=400
)

In [5]:
# Create a selection that choose the nearest point & selects based on Start Date of sso
nearest = alt.selection(type='single', nearest=True, on='mouseover',
                       fields=['Start Date'], empty='none')

In [6]:
# Manipulate the start date to represent only one times in gantt chart if there are two overflows in a day
unique_date = set(df_sso['Start Date'].values.tolist())
column_date = []

for date in df_sso['Start Date'].values.tolist():
    if date in unique_date:
        column_date.append(date)
        unique_date.remove(date)
    else:
        column_date.append(' ')
        
df_sso['Date'] = column_date
df_sso.head(5)

,SSO Location,Neighborhood,Spill Type,Start Date,Start Time,End Date,End Time,Spill Volume,Cause,Corrective Action,Latitude,Longitude,Date
26,254 SUMMER ST,SOUTH BOSTON,PUBLIC,10/18/2021,2022-12-05 09:12:00,10/18/2021,2022-12-05 11:45:00,30,FOG,FURTHER INVESTIGATION REQUIRED,42.390062,-71.113337,10/18/2021
7,1455 COMMONWEALTH AV,ALLSTON/BRIGHTON,PRIVATE - 100 GALLONS OR MORE,10/23/2021,2022-12-05 19:52:00,10/24/2021,2022-12-05 14:30:00,1000,PRIVATE LATERAL,OWNER RESPONSIBILITY,42.348773,-71.141198,10/23/2021
8,1455 COMMONWEALTH AV,ALLSTON/BRIGHTON,PRIVATE - NOT CONTAINED TO BUILDING,10/23/2021,2022-12-05 19:52:00,10/24/2021,2022-12-05 14:30:00,1000,PRIVATE LATERAL,OWNER RESPONSIBILITY,42.348773,-71.141198,
2,125 NASHUA ST,CENTRAL,PUBLIC,12/14/2021,2022-12-05 09:51:00,12/14/2021,2022-12-05 11:35:00,50,DEBRIS,FURTHER INVESTIGATION REQUIRED,42.367100,-71.064000,12/14/2021
43,512 CENTRE ST,JAMAICA PLAIN,PUBLIC,12/15/2021,2022-12-05 10:00:00,12/15/2021,2022-12-05 12:15:00,300,DEBRIS,FURTHER INVESTIGATION REQUIRED,42.317590,-71.112659,12/15/2021


In [9]:
# x-axis date range from 2021-10-18 to 2022-10-5
domain_pd = pd.to_datetime(["2021-10-18","2022-10-5"]).astype(int) / 10 ** 6

# The basic scatter plot
scatters = alt.Chart(df_sso, title="Spill Volume of Sanitary Sewer Overflow").mark_point(color='red').encode(
    x=alt.X('Start Date:T', scale=alt.Scale(domain=list(domain_pd))),
    y='Spill Volume:Q',
    opacity=alt.condition(nearest, alt.value(1), alt.value(0.2))
).properties(
    width=800,
    height=200
)

# Transparent selectors accross the chart. This is what tell us the x-value of the cursor
selectors = alt.Chart(df_sso).mark_point().encode(
    x=alt.X('Start Date:T', scale=alt.Scale(domain=list(domain_pd))),
    opacity=alt.value(0)
).add_selection(
     nearest
).properties(
    width=600,
    height=200
)

# Text to represent the spill volum when the point selected
text = scatters.mark_text(align='left', dx=5, dy=-5).encode(
    text=alt.condition(nearest, 'Spill Volume:Q', alt.value(' '))
)

# gray verticle line in the graph
rules = alt.Chart(df_sso).mark_rule(color='gray').encode(
    x=alt.X('Start Date:T', scale=alt.Scale(domain=list(domain_pd))),
).transform_filter(
    nearest
)

In [10]:
# Screen 2 Timeline (Add all components to make timeline chart)
timeline = alt.layer(
    scatters, selectors, rules, text
)

In [11]:
# Points for the overflow accidents' location
map_sso_points = alt.Chart(df_sso).mark_circle(
    color='red'
).encode(
    latitude="Latitude:Q",
    longitude="Longitude:Q",
    size=alt.value(20),
    opacity=alt.condition(nearest, alt.value(1), alt.value(0))
).properties(
    width=400,
    height=400
)

# Create New Column the format SSO Location + " " + Neighborhood
df_sso['Location Neighborhood'] = df_sso[['SSO Location', 'Neighborhood']].agg(' '.join, axis=1)

# Text for address
map_sso_text = alt.Chart(df_sso).mark_text(align='left', dx=-30, dy=10, fontSize=10).encode(
    latitude="Latitude:Q",
    longitude="Longitude:Q",
    text=alt.condition(nearest, 'Location Neighborhood:N', alt.value(' '))
)

sso_map = alt.layer(
    background, map_sso_points, map_sso_text
)

In [12]:
# Gantt Chart 
gantt_chart = alt.Chart(df_sso, title="Start Time and End Time of SSO").mark_bar().encode(
    x = 'Start Time:T',
    x2 = 'End Time:T',
    y = 'Start Date:T',
    opacity=alt.condition(nearest, alt.value(1), alt.value(0))
).properties(
    width=400,
    height=400
)

# The text is shown up by the selection from timeline chart
gantt_date_text = gantt_chart.mark_text(align='left', dx=0, dy=-10).encode(
    text=alt.condition(nearest, 'Date', alt.value(' '))
)

# Add gantt chart and text
gantt = alt.layer(
    gantt_chart, gantt_date_text
)

In [13]:
chart1 = alt.vconcat(alt.hconcat(sso_map, gantt), timeline, center=True)
chart1

alt.VConcatChart(...)

In [14]:
# Static Screen 3
neighbor_selection = alt.selection_multi(fields=['Site'])
legend_selection = alt.selection_multi(fields=['Site'], bind='legend')

year = alt.Chart(df_rain_year).mark_bar().encode(
    x='Site',
    y='Inches',
    color = alt.Color(field="Site", type="nominal", scale=alt.Scale(scheme='category10'))
).properties(
    title='Boston Neighborhoods Annual Rainfall in 2022'
).add_selection(
    neighbor_selection
)

month = alt.Chart(df_rain_month).mark_line().encode(
    x=alt.X('Month', type='nominal', sort=None),
    y=alt.Y('Inches', scale=alt.Scale(domain=[0,6])),
    color = alt.Color(field="Site", type="nominal", scale=alt.Scale(scheme='category10')),
    opacity=alt.condition(legend_selection, alt.value(1), alt.value(0.2))
).properties(
    title='Boston Neighborhoods Monthly Rainfall in 2022'
).transform_filter(
    neighbor_selection
).add_selection(
    legend_selection
)

chart2 = year | month
chart2

alt.HConcatChart(...)

In [125]:
# Save the chart html format
chart1.save('chart1.html')
chart2.save('chart2.html')